# Imputing X values

In [2]:
import numpy as np
import pandas as pd
import anndata as ad

## Prepare data

In [3]:
# Metadata table
human = pd.read_excel('/mnt/home/wkoh1/ceph/GSE97179/human.xlsx', skiprows=1)

/mnt/home/wkoh1/miniconda3/envs/scaden/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
human.head()

,Sample,Age,FACS date,Brain area,Laminar layer,Labeling,FACS channel,FACS count,Bisulfite conversion method,Library type,...,% Filtered reads,mCCC/CCC,mCG/CG,mCH/CH,Estimated mCG/CG,Estimated mCH/CH,Coverage (%),Neuron type,tSNE x coordinate,tSNE y coordinate
0,Pool_1_AD008_indexed,25 yr,2016-06-30,Mid Frontal Gyrus,NaN,Anti-NeuN,488+,1,Zymo EZ-96 direct,snmC-seq,...,0.531,0.01291,0.78173,0.06637,0.77888,0.05416,3.58,hL2/3,-12.332700,-11.750700
1,Pool_1_AD010_indexed,25 yr,2016-06-30,Mid Frontal Gyrus,NaN,Anti-NeuN,488+,1,Zymo EZ-96 direct,snmC-seq,...,0.541,0.00812,0.76726,0.04277,0.76535,0.03493,4.17,hL2/3,-11.088300,11.283700
2,Pool_2_AD008_indexed,25 yr,2016-06-30,Mid Frontal Gyrus,NaN,Anti-NeuN,488+,1,Zymo EZ-96 direct,snmC-seq,...,0.526,0.00844,0.79328,0.04561,0.79152,0.03749,5.55,hL4,-0.698722,-0.094122
3,Pool_2_AD010_indexed,25 yr,2016-06-30,Mid Frontal Gyrus,NaN,Anti-NeuN,488+,1,Zymo EZ-96 direct,snmC-seq,...,0.531,0.01006,0.77232,0.05277,0.77001,0.04314,4.80,hL2/3,-15.662800,5.273410
4,Pool_3_AD008_indexed,25 yr,2016-06-30,Mid Frontal Gyrus,NaN,Anti-NeuN,488+,1,Zymo EZ-96 direct,snmC-seq,...,0.585,0.00842,0.82762,0.04547,0.82616,0.03736,3.54,hSst-2,14.084100,-4.481540


In [5]:
# Matrix containing the methylation values
data = ad.read_h5ad('/mnt/home/wkoh1/ceph/GSE97179/cells_exact_strand/matrix.h5ad').to_df()
data.head()

,Pool_258_AD010_indexed,Pool_240_AD010_indexed,Pool_402_AD008_indexed,Pool_495_AD006_indexed,Pool_1101_AD006_indexed,Pool_990_AD008_indexed,Pool_1416_AD006_indexed,Pool_1484_AD002_indexed,Pool_207_AD008_indexed,Pool_215_AD008_indexed,...,Pool_994_AD010_indexed,Pool_1058_AD006_indexed,Pool_1469_AD008_indexed,Pool_659_AD010_indexed,Pool_229_AD010_indexed,Pool_1339_AD010_indexed,Pool_964_AD008_indexed,Pool_1393_AD006_indexed,Pool_809_AD008_indexed,Pool_191_AD008_indexed
name,,,,,,,,,,,,,,,,,,,,,
cg06154531,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg03269716,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg16349774,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg11782208,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
cg14252395,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.shape

(241580, 2784)

In [7]:
# Table of average non-NA methylation values by cell type
bulk_df = pd.read_csv('/mnt/home/wkoh1/ceph/GSE97179/bulk.tsv', sep='\t', index_col='name')
bulk_df.head()

,hL2/3,hL4,hSst-2,hDL-1,hL5-2,hVip-2,hNos,hL5-4,hL5-3,hL6-2,...,hDL-2,hVip-1,hPv-1,hNdnf,hSst-1,hSst-3,hDL-3,hL6-1,hL5-1,hPv-2
name,,,,,,,,,,,,,,,,,,,,,
cg06154531,0.972973,1.000000,1.000000,1.000,1.00,1.0,NaN,1.000000,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,NaN,1.000000,1.0,1.0,1.000000
cg03269716,0.956522,1.000000,0.900000,1.000,1.00,1.0,1.0,0.500000,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,0.75,0.833333,1.0,1.0,0.666667
cg16349774,0.851852,0.833333,0.714286,0.875,1.00,1.0,1.0,0.857143,NaN,0.818182,...,0.8,0.5,1.000000,1.0,1.0,NaN,1.000000,1.0,1.0,NaN
cg11782208,0.941176,1.000000,1.000000,1.000,1.00,1.0,1.0,0.888889,1.0,1.000000,...,0.8,1.0,0.909091,1.0,1.0,1.00,1.000000,1.0,1.0,NaN
cg14252395,0.975610,0.916667,0.875000,1.000,0.75,1.0,1.0,1.000000,1.0,1.000000,...,1.0,1.0,1.000000,0.9,1.0,1.00,1.000000,1.0,1.0,1.000000


In [8]:
bulk_df.shape

(241580, 21)

In [10]:
# Impute the missing values in bulk_df
imputed = pd.read_csv('/mnt/home/wkoh1/ceph/GSE97179/pseudo_bulk/GSE97179_pseudo_bulk_var0.15_imputed.tsv', sep='\t')
imputed.head()

,hL2.3,hL4,hSst.2,hDL.1,hL5.2,hVip.2,hNos,hL5.4,hL5.3,hL6.2,...,hDL.2,hVip.1,hPv.1,hNdnf,hSst.1,hSst.3,hDL.3,hL6.1,hL5.1,hPv.2
cg10375954,0.000000,0.000000,0.999974,0.000000,0.000000,0.666667,0.999999,0.500000,0.750000,0.00,...,0.000000,0.000000,0.200000,0.666667,1.0,1.0,0.000008,0.666667,0.0,0.999999
cg11157076,0.833333,0.000004,1.000000,0.750000,0.000004,1.000000,0.000000,0.333333,0.023167,0.00,...,1.000000,0.333333,0.833333,0.999925,1.0,1.0,1.000000,1.000000,1.0,0.000000
cg21950720,0.049180,0.111111,0.892857,0.000000,0.000000,0.636364,0.666667,0.050000,0.000000,0.00,...,0.000000,0.500000,0.875000,0.444444,1.0,1.0,0.000000,0.000000,0.0,1.000000
cg19224645,0.000000,0.000004,1.000000,0.000000,0.500000,1.000000,0.999999,0.000000,0.023167,0.00,...,0.000000,1.000000,0.000000,0.500000,1.0,1.0,1.000000,1.000000,0.0,0.000000
cg14977608,0.152542,0.142857,1.000000,0.357143,0.500000,1.000000,1.000000,0.083333,0.000000,0.75,...,0.166667,1.000000,1.000000,1.000000,1.0,1.0,0.250000,0.500000,0.0,1.000000


In [11]:
imputed.shape

(9007, 21)

In [12]:
# Get the cell type of every sample
neuron_type = pd.DataFrame(human['Neuron type'])
neuron_type.set_index(human['Sample'], inplace=True)
neuron_type

,Neuron type
Sample,
Pool_1_AD008_indexed,hL2/3
Pool_1_AD010_indexed,hL2/3
Pool_2_AD008_indexed,hL4
Pool_2_AD010_indexed,hL2/3
Pool_3_AD008_indexed,hSst-2
...,...
Pool_1487_AD010_indexed,hSst-2
Pool_1488_AD002_indexed,hPv-1
Pool_1488_AD006_indexed,hL6-2


In [13]:
# Make sure that the column names match between the imputed and the bulk_df
a = pd.unique(neuron_type['Neuron type'])
print(a)
imputed.columns = a[pd.notna(a)]

['hL2/3' 'hL4' 'hSst-2' 'hDL-1' 'hL5-2' 'hVip-2' 'hNos' 'hL5-4' 'hL5-3'
 'hL6-2' 'hL6-3' 'hDL-2' 'hVip-1' 'hPv-1' 'hNdnf' 'hSst-1' 'hSst-3'
 'hDL-3' 'hL6-1' 'hL5-1' nan 'hPv-2']


In [14]:
# Filter the X matrix for the CpGs that pass the variance filter
x = data.loc[imputed.index]
x

,Pool_258_AD010_indexed,Pool_240_AD010_indexed,Pool_402_AD008_indexed,Pool_495_AD006_indexed,Pool_1101_AD006_indexed,Pool_990_AD008_indexed,Pool_1416_AD006_indexed,Pool_1484_AD002_indexed,Pool_207_AD008_indexed,Pool_215_AD008_indexed,...,Pool_994_AD010_indexed,Pool_1058_AD006_indexed,Pool_1469_AD008_indexed,Pool_659_AD010_indexed,Pool_229_AD010_indexed,Pool_1339_AD010_indexed,Pool_964_AD008_indexed,Pool_1393_AD006_indexed,Pool_809_AD008_indexed,Pool_191_AD008_indexed
cg10375954,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg11157076,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg21950720,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg19224645,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
cg14977608,1.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg08845336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg03958705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg09219688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg15527515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Find and remove samples with missing labels
print(human[pd.isna(human['Neuron type'])]['Sample'].shape)
print(human[pd.isna(human['Neuron type'])]['Sample'].head())

(44,)
152     Pool_83_AD008_indexed
175     Pool_95_AD010_indexed
230    Pool_126_AD008_indexed
322    Pool_176_AD010_indexed
325    Pool_178_AD010_indexed
Name: Sample, dtype: object


In [16]:
x = x.drop(columns=human[pd.isna(human['Neuron type'])]['Sample'])
x

,Pool_258_AD010_indexed,Pool_240_AD010_indexed,Pool_402_AD008_indexed,Pool_495_AD006_indexed,Pool_1101_AD006_indexed,Pool_990_AD008_indexed,Pool_1416_AD006_indexed,Pool_1484_AD002_indexed,Pool_207_AD008_indexed,Pool_215_AD008_indexed,...,Pool_1320_AD002_indexed,Pool_994_AD010_indexed,Pool_1058_AD006_indexed,Pool_1469_AD008_indexed,Pool_229_AD010_indexed,Pool_1339_AD010_indexed,Pool_964_AD008_indexed,Pool_1393_AD006_indexed,Pool_809_AD008_indexed,Pool_191_AD008_indexed
cg10375954,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg11157076,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg21950720,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg19224645,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
cg14977608,1.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg08845336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg03958705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg09219688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg15527515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Impute step

In [17]:
# Sample from binomial distribution, with probability 
def impute_val(col, rng):
    for n in col.index: # cpg
        if pd.isna(col[n]):
            col[n] = rng.binomial(n = 1, p = imputed.at[n, neuron_type.at[col.name, 'Neuron type']])
    return col

In [18]:
rng = np.random.default_rng(seed = 10)
imputed_x = x.apply(impute_val, rng=rng)

In [19]:
imputed_x.to_csv('/mnt/home/wkoh1/ceph/GSE97179/cells_exact_strand/imputed_x2.tsv', sep='\t')

In [20]:
imputed_x

,Pool_258_AD010_indexed,Pool_240_AD010_indexed,Pool_402_AD008_indexed,Pool_495_AD006_indexed,Pool_1101_AD006_indexed,Pool_990_AD008_indexed,Pool_1416_AD006_indexed,Pool_1484_AD002_indexed,Pool_207_AD008_indexed,Pool_215_AD008_indexed,...,Pool_1320_AD002_indexed,Pool_994_AD010_indexed,Pool_1058_AD006_indexed,Pool_1469_AD008_indexed,Pool_229_AD010_indexed,Pool_1339_AD010_indexed,Pool_964_AD008_indexed,Pool_1393_AD006_indexed,Pool_809_AD008_indexed,Pool_191_AD008_indexed
cg10375954,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
cg11157076,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
cg21950720,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
cg19224645,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
cg14977608,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg08845336,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
cg03958705,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
cg09219688,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
cg15527515,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
